In [1]:
from openai import OpenAI
import sys
import io
import traceback


In [2]:
client = OpenAI(
base_url = 'http://localhost:11434/v1',
api_key='ollama', # required, but unused
)

In [3]:
def query_classifier(query):
    prompt = f"Classify the following query as either 'Informational' or 'Coding'. : {query}"
    response = client.chat.completions.create(
    model='qwen2.5:14b-instruct-q5_K_M',  # Adjust to the model you are using
        messages=[{"role": "system", "content": """You are a helpful assistant helping classification task.
        You will analyze the query and Reply with only Informational or Coding. Note Coding only if it is Python Coding"""},
                  {"role": "user", "content": prompt},
                 ]
    )
    classification = response.choices[0].message.content.strip()
    print(classification)
    return classification


In [4]:
def informational_response(query):
    print("Starting informational_response...")
    response = client.chat.completions.create(
    model='qwen2.5:14b-instruct-q5_K_M',
        messages=[{"role": "system", "content": 'You are a helpful assistant'},
                  {"role": "user", "content": query}]
    )
    return response.choices[0].message.content.strip()


In [5]:
def coding_response(query):
    print("Starting coding_response...")
    response = client.chat.completions.create(
    model='qwen2.5:14b-instruct-q5_K_M',
        messages=[{"role": "system", "content": """You are a coding assistant that executes python code. 
                   Your response will be sent to code execution, so strictly generate code only. Use print statements to display output appropriately
                   Anything you want to say, say it in single line comments"""},
            {"role": "user", "content": f"Generate Python code for: {query}"}]
    )
    code = response.choices[0].message.content.strip()
    return {"file_name": "generated_code.py", "code": code}


In [6]:
def execute_code(code):
    try:
        # Define the code and execute it
        exec(code)
        return locals(), None  # Return the local variables and None for error
    except Exception as e:
        return None, f"Error: {str(e)}"  # Return None for output and error message

In [7]:
def get_fixed_code(code, error):
    print("Starting get_fixed_code...")
    prompt = f"Here is some Python code that produced an error:\n\n{code}\n\nError message: {error}\n\nPlease provide a corrected version of the code."
    response = client.chat.completions.create(
    model='qwen2.5:14b-instruct-q5_K_M',
        messages=[{"role": "system", "content": "You are a coding assistant that assists in Python coding tasks."},
            {"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()


In [9]:
def execute_code_until_success(code):
    print("Starting execute_code_until_success...")
    attempts = 0
    while attempts < 5:
        print(f"Attempting to execute code:\n```python\n{code}\n```")  # Debug output
        print("Output")
        output, error = execute_code(code)  # Unpacking the returned tuple
        if output is not None:
            return output  # Return the local variables if execution was successful
        else:
            print(f"Attempt {attempts + 1} failed with error: {error}")
            code = get_fixed_code(code, error)  # Get a fixed version of the code
            attempts += 1
    return "Code could not be executed successfully after 5 attempts."


In [11]:
def main():
    user_query = input("Enter your query: ")
    classification = query_classifier(user_query)

    if classification == "Informational":
        answer = informational_response(user_query)
        print(f"Answer: {answer}")
    elif classification == "Coding":
        coding_info = coding_response(user_query)
        output = execute_code_until_success(coding_info["code"])
        #print(f"Output: {output}")
    else:
        print("Could not classify the query.")

if __name__ == "__main__":
    main()


Enter your query: What is the capital of Norway?
Informational
Starting informational_response...
Answer: The capital of Norway is Oslo.
